In [1]:
pip install nltk rouge-score bert-score

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 30.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 24.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 34.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 50.5 MB/s eta 0:00:00
  Created wheel for rouge-score: filename=rouge_score-0.1.2-py3-none-any.whl size=249

In [2]:
ref_summary="""
The German Johannes Gutenberg introduced printing in Europe. His invention had a decisive contribution in spread of mass-learning and in building the basis of the modern society.
Gutenberg major invention was a practical system permitting the mass production of printed books. The printed books allowed open circulation of information, and prepared the evolution of society from to the contemporary knowledge-based economy.
"""

In [3]:
import re

def count_words(text):
  """Counts the number of words in a given string.

  Args:
    text: The input string.

  Returns:
    The number of words in the string.
  """
  words = re.findall(r'\b\w+\b', text)  # Find all words using regular expressions
  return len(words)

word_count = count_words(ref_summary)
print(f"Number of words in the string: {word_count}")

Number of words in the string: 63


In [4]:
from rouge_score import rouge_scorer
from bert_score import score as bert_score

def calculate_summary_scores(reference_text, generated_summary):
    # Prepare inputs
    reference_list = [reference_text.split()]
    generated_list = generated_summary.split()

    # ROUGE Scores
    rouge_scorer_instance = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
    rouge_scores = rouge_scorer_instance.score(reference_text, generated_summary)

    # BERTScore
    P, R, F1 = bert_score([generated_summary], [reference_text], lang="en")

    # Display results
    print("Summary Evaluation Metrics:")
    print("-" * 30)
    print("\nROUGE Scores:")
    print(f"  ROUGE-1: Precision: {rouge_scores['rouge1'].precision:.4f}, Recall: {rouge_scores['rouge1'].recall:.4f}, F1: {rouge_scores['rouge1'].fmeasure:.4f}")
    print(f"  ROUGE-2: Precision: {rouge_scores['rouge2'].precision:.4f}, Recall: {rouge_scores['rouge2'].recall:.4f}, F1: {rouge_scores['rouge2'].fmeasure:.4f}")
    print(f"  ROUGE-L: Precision: {rouge_scores['rougeL'].precision:.4f}, Recall: {rouge_scores['rougeL'].recall:.4f}, F1: {rouge_scores['rougeL'].fmeasure:.4f}")
    print("\nBERT Score:")
    print(f"  Precision: {P.mean().item():.4f}")
    print(f"  Recall: {R.mean().item():.4f}")
    print(f"  F1 Score: {F1.mean().item():.4f}")

## References:

https://huggingface.co/spaces/evaluate-metric/bertscore

In [5]:
text = """Johannes Gutenberg was a German goldsmith and publisher who introduced printing to Europe. His introduction of mechanical movable type printing to Europe started the Printing Revolution and is widely regarded as the most important event of the modern period. It played a key role in the scientific revolution and laid the basis for the modern knowledge-based economy and the spread of learning to the masses.

Gutenberg many contributions to printing are: the invention of a process for mass-producing movable type, the use of oil-based ink for printing books, adjustable molds, and the use of a wooden printing press. His truly epochal invention was the combination of these elements into a practical system that allowed the mass production of printed books and was economically viable for printers and readers alike.

In Renaissance Europe, the arrival of mechanical movable type printing introduced the era of mass communication which permanently altered the structure of society. The relatively unrestricted circulation of information—including revolutionary ideas—transcended borders, and captured the masses in the Reformation. The sharp increase in literacy broke the monopoly of the literate elite on education and learning and bolstered the emerging middle class.

"""

In [6]:
word_count = count_words(ref_summary)
print(f"Number of words in the string: {word_count}")

Number of words in the string: 63


**Google T5**

In [7]:
import torch

from transformers import AutoTokenizer, AutoModelWithLMHead

In [8]:
tokenizer = AutoTokenizer.from_pretrained('t5-base')
model = AutoModelWithLMHead.from_pretrained('t5-base', return_dict=True)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

/usr/local/lib/python3.11/dist-packages/transformers/models/auto/modeling_auto.py:1838: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language models, `AutoModelForMaskedLM` for masked language models and `AutoModelForSeq2SeqLM` for encoder-decoder models.
  warnings.warn(


model.safetensors:   0%|          | 0.00/892M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

**Prompt 1**

In [9]:
inputs = tokenizer.encode("Summarize :   " + text,
return_tensors='pt',
max_length=512,
truncation=True)

In [10]:
summary_ids = model.generate(inputs, max_length=100, min_length=60)
summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)

In [11]:
summary

"Gutenberg's inventions include: adjustable molds, adjustable molds, and a wooden printing press. Gutenberg's inventions include: adjustable molds, adjustable molds, and adjustable molds. the rise of mass communication and the spread of knowledge to the masses. The rise of literacy and the spread of knowledge to the masses fueled the Reformation. Gutenberg's printing revolution was a key event in the scientific revolution and is widely regarded"

In [12]:
print("Scores for T5 Prompt 1")
calculate_summary_scores(text, summary)

Scores for T5 Prompt 1


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/482 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.42G [00:00<?, ?B/s]

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Summary Evaluation Metrics:
------------------------------

ROUGE Scores:
  ROUGE-1: Precision: 0.7353, Recall: 0.2564, F1: 0.3802
  ROUGE-2: Precision: 0.3731, Recall: 0.1289, F1: 0.1916
  ROUGE-L: Precision: 0.4265, Recall: 0.1487, F1: 0.2205

BERT Score:
  Precision: 0.8962
  Recall: 0.8499
  F1 Score: 0.8724


**Prompt 2**

In [13]:
inputs = tokenizer.encode("Summarize and give as key points: " + text,
return_tensors='pt',
max_length=512,
truncation=True)

In [14]:
summary_ids = model.generate(inputs, max_length=80, min_length=60)
summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)

In [15]:
summary

'movable type printing, adjustable molds, and a wooden printing press. Johannes Gutenberg the main points: the main points: Johannes Gutenberg the main points: Johannes Gutenberg. and. Johannes Gutenberg. the printing revolution. and the printing revolution. Gutenberg the printing revolution. the'

In [16]:
print("Scores for T5 Prompt 2")
calculate_summary_scores(text, summary)

Scores for T5 Prompt 2


Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Summary Evaluation Metrics:
------------------------------

ROUGE Scores:
  ROUGE-1: Precision: 0.6750, Recall: 0.1385, F1: 0.2298
  ROUGE-2: Precision: 0.3077, Recall: 0.0619, F1: 0.1030
  ROUGE-L: Precision: 0.4750, Recall: 0.0974, F1: 0.1617

BERT Score:
  Precision: 0.8524
  Recall: 0.8150
  F1 Score: 0.8332


# Hugging Face

In [17]:
from transformers import pipeline

**DistilBERT**

In [18]:

summarizer = pipeline("summarization", model="sshleifer/distilbart-cnn-12-6")
# Generate the summary
summary = summarizer(text, max_length=80, min_length=60, do_sample=False)
# Print the summary
summary=summary[0]['summary_text']

config.json:   0%|          | 0.00/1.80k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.22G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

Device set to use cpu


In [19]:
summary

' Johannes Gutenberg was a German goldsmith and publisher who introduced printing to Europe . His introduction of mechanical movable type printing is widely regarded as the most important event of the modern period . It played a key role in the scientific revolution and laid the basis for the modern knowledge-based economy and the spread of learning to the masses .'

In [20]:
print("Scores for DistilBERT")
calculate_summary_scores(text, summary)

Scores for DistilBERT


Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Summary Evaluation Metrics:
------------------------------

ROUGE Scores:
  ROUGE-1: Precision: 1.0000, Recall: 0.3026, F1: 0.4646
  ROUGE-2: Precision: 0.9828, Recall: 0.2938, F1: 0.4524
  ROUGE-L: Precision: 1.0000, Recall: 0.3026, F1: 0.4646

BERT Score:
  Precision: 0.9690
  Recall: 0.8774
  F1 Score: 0.9209


**T5 Small**

In [21]:
summarizer = pipeline("summarization", model="t5-small")
# Generate the summary
summary = summarizer(text, max_length=80, min_length=60, do_sample=False)
# Print the summary
summary=summary[0]['summary_text']

config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

Device set to use cpu


In [22]:
summary

'German goldsmith and publisher Johannes Gutenberg introduced printing to the world . he was a pioneer in the printing revolution . his invention of a process for mass-producing movable type was widely regarded as the most important event of the modern period . it played a key role in the scientific revolution and laid the basis for the modern knowledge-based economy .'

In [23]:
print("Scores for T5 Small")
calculate_summary_scores(text, summary)

Scores for T5 Small


Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Summary Evaluation Metrics:
------------------------------

ROUGE Scores:
  ROUGE-1: Precision: 0.9500, Recall: 0.2923, F1: 0.4471
  ROUGE-2: Precision: 0.8136, Recall: 0.2474, F1: 0.3794
  ROUGE-L: Precision: 0.6833, Recall: 0.2103, F1: 0.3216

BERT Score:
  Precision: 0.9385
  Recall: 0.8671
  F1 Score: 0.9014


**PEGASUS**

In [24]:
summarizer = pipeline("summarization", model="google/pegasus-cnn_dailymail")
# Generate the summary
summary = summarizer(text, max_length=100, min_length=90, do_sample=False)
# Print the summary
summary=summary[0]['summary_text']

config.json:   0%|          | 0.00/1.12k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/2.28G [00:00<?, ?B/s]

Some weights of PegasusForConditionalGeneration were not initialized from the model checkpoint at google/pegasus-cnn_dailymail and are newly initialized: ['model.decoder.embed_positions.weight', 'model.encoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


generation_config.json:   0%|          | 0.00/280 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/88.0 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/1.91M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/65.0 [00:00<?, ?B/s]

Device set to use cpu


In [25]:
summary

'Johannes Gutenberg was a German goldsmith and publisher who introduced printing to Europe .<n>His introduction of mechanical movable type printing to Europe started the Printing Revolution and is widely regarded as the most important event of the modern period .<n>The arrival of mechanical movable type printing introduced the era of mass communication which permanently altered the structure of society .<n>The sharp increase in literacy broke the monopoly of the literate elite on education and learning and bolstered the emerging middle class .'

In [26]:
print("Scores for Pegasus")
calculate_summary_scores(text, summary)

Scores for Pegasus


Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Summary Evaluation Metrics:
------------------------------

ROUGE Scores:
  ROUGE-1: Precision: 0.9643, Recall: 0.4154, F1: 0.5806
  ROUGE-2: Precision: 0.9277, Recall: 0.3969, F1: 0.5560
  ROUGE-L: Precision: 0.9643, Recall: 0.4154, F1: 0.5806

BERT Score:
  Precision: 0.9398
  Recall: 0.8884
  F1 Score: 0.9134


 **LED (Longformer Encoder-Decoder)**

In [27]:
summarizer = pipeline("summarization", model="allenai/led-base-16384")
# Generate the summary
summary = summarizer(text, max_length=100, min_length=90, do_sample=False)
# Print the summary
summary=summary[0]['summary_text']

config.json:   0%|          | 0.00/1.09k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/648M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/168 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/27.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/772 [00:00<?, ?B/s]

Device set to use cpu
Input ids are automatically padded from 235 to 1024 to be a multiple of `config.attention_window`: 1024


In [28]:
summary

'Johannes Gutenberg was a German goldsmith and publisher who introduced printing to Europe. His introduction of mechanical movable type printing to Europe started the Printing Revolution and is widely regarded as the most important event of the modern period. It played a key role in the scientific revolution and laid the basis for the modern knowledge-based economy and the spread of learning to the masses.Johannes Gutenberg [ edit ]Gutenberg many contributions to printing are: the invention of a process for mass'

In [29]:
print("Scores for LED")
calculate_summary_scores(text, summary)

Scores for LED


Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Summary Evaluation Metrics:
------------------------------

ROUGE Scores:
  ROUGE-1: Precision: 0.9634, Recall: 0.4051, F1: 0.5704
  ROUGE-2: Precision: 0.9506, Recall: 0.3969, F1: 0.5600
  ROUGE-L: Precision: 0.9634, Recall: 0.4051, F1: 0.5704

BERT Score:
  Precision: 0.9578
  Recall: 0.8904
  F1 Score: 0.9229


**BART**

In [30]:
text

'Johannes Gutenberg was a German goldsmith and publisher who introduced printing to Europe. His introduction of mechanical movable type printing to Europe started the Printing Revolution and is widely regarded as the most important event of the modern period. It played a key role in the scientific revolution and laid the basis for the modern knowledge-based economy and the spread of learning to the masses.\n\nGutenberg many contributions to printing are: the invention of a process for mass-producing movable type, the use of oil-based ink for printing books, adjustable molds, and the use of a wooden printing press. His truly epochal invention was the combination of these elements into a practical system that allowed the mass production of printed books and was economically viable for printers and readers alike.\n\nIn Renaissance Europe, the arrival of mechanical movable type printing introduced the era of mass communication which permanently altered the structure of society. The relativ

In [31]:
summarizer = pipeline("summarization", model="facebook/bart-large-xsum")
# Generate the summary
summary = summarizer(text, max_length=90, min_length=70, do_sample=False)
# Print the summary
summary=summary[0]['summary_text']

config.json:   0%|          | 0.00/1.51k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/309 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Device set to use cpu


In [32]:
summary

'Johannes Gutenberg was one of the most important inventors of the 19th Century and is widely regarded as the founder of printing in Europe and the United States, as well as a key figure in the development of the printing press. and its use in the printing industry in the world of science, technology, and communication. and the printing revolution.'

In [33]:
print("Scores for BART X-SUM")
calculate_summary_scores(text, summary)

Scores for BART X-SUM


Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Summary Evaluation Metrics:
------------------------------

ROUGE Scores:
  ROUGE-1: Precision: 0.7167, Recall: 0.2205, F1: 0.3373
  ROUGE-2: Precision: 0.3390, Recall: 0.1031, F1: 0.1581
  ROUGE-L: Precision: 0.5167, Recall: 0.1590, F1: 0.2431

BERT Score:
  Precision: 0.8909
  Recall: 0.8393
  F1 Score: 0.8643


In [34]:
summarizer = pipeline("summarization", model="facebook/bart-large-cnn")
# Generate the summary
summary = summarizer(text, max_length=80, min_length=60, do_sample=False)
# Print the summary
summary=summary[0]['summary_text']

config.json:   0%|          | 0.00/1.58k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Device set to use cpu


In [35]:
summary

'Johannes Gutenberg was a German goldsmith and publisher. His introduction of mechanical movable type printing to Europe started the Printing Revolution. It played a key role in the scientific revolution and laid the basis for the modern knowledge-based economy and the spread of learning to the masses. The relatively unrestricted circulation of information—including revolutionary ideas—transcended borders, and captured the masses'

In [36]:
print("Scores for BART CNN Data")
calculate_summary_scores(text, summary)

Scores for BART CNN Data


Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Summary Evaluation Metrics:
------------------------------

ROUGE Scores:
  ROUGE-1: Precision: 1.0000, Recall: 0.3231, F1: 0.4884
  ROUGE-2: Precision: 0.9516, Recall: 0.3041, F1: 0.4609
  ROUGE-L: Precision: 1.0000, Recall: 0.3231, F1: 0.4884

BERT Score:
  Precision: 0.9655
  Recall: 0.8838
  F1 Score: 0.9229


**PEGASUS**

In [37]:
summarizer = pipeline("summarization", model="google/pegasus-cnn_dailymail")
# Generate the summary
summary = summarizer(text, max_length=100, min_length=90, do_sample=False)
# Print the summary
summary=summary[0]['summary_text']

Some weights of PegasusForConditionalGeneration were not initialized from the model checkpoint at google/pegasus-cnn_dailymail and are newly initialized: ['model.decoder.embed_positions.weight', 'model.encoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Device set to use cpu


In [38]:
summary

'Johannes Gutenberg was a German goldsmith and publisher who introduced printing to Europe .<n>His introduction of mechanical movable type printing to Europe started the Printing Revolution and is widely regarded as the most important event of the modern period .<n>The arrival of mechanical movable type printing introduced the era of mass communication which permanently altered the structure of society .<n>The sharp increase in literacy broke the monopoly of the literate elite on education and learning and bolstered the emerging middle class .'

In [39]:
print("Scores for PEGASUS")
calculate_summary_scores(text, summary)

Scores for PEGASUS


Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Summary Evaluation Metrics:
------------------------------

ROUGE Scores:
  ROUGE-1: Precision: 0.9643, Recall: 0.4154, F1: 0.5806
  ROUGE-2: Precision: 0.9277, Recall: 0.3969, F1: 0.5560
  ROUGE-L: Precision: 0.9643, Recall: 0.4154, F1: 0.5806

BERT Score:
  Precision: 0.9398
  Recall: 0.8884
  F1 Score: 0.9134
